As attempted in [solver.ipynb](solver.ipynb), informed search can perform relatively well. I must say, this is quite an achievement after CS2040S and CS2109S. However, it takes forever to solve some configurations. Hence, in this notebook, I will be attempting a neural network solution at finding the shortest solution to Rubik's cube.

## Rubik's cube convention

To recap, Rubik's cube has the following index convention:

![Rubik indices](images/face-indices.png)

Given a state in numpy array of dimension 6 x 3 x 3, the index corresponds to the face indicated above. The central face in the image is the front face.
Each block is then a 3 x 3 array that corresponds to the cells.

Each cell colour is indicated by an integer between 0 to 5 (inclusive).

The correspondences between indices and faces are:
* 0: face up (U)
* 1: face left (L)
* 2: face front (F)
* 3: face right (R)
* 4: face down (D)
* 5: face back (B)

Each action is one of `"L"`, `"L'"`, `"R"`, `"R'"`, `"F"`, `"F'"`, `"B"`, `"B'"`, `"U"`, `"U'"`, `"D"`, `"D'"`, where action without apostrophe `'` is clockwise and action with apostrophe `'` is anticlockwise. Clockwise direction is determined by the rotation of the face.

## Neural network

The neural network is designed as such: given a Rubik's cube position, return the best move for that position. Hence the neural network will take 54 inputs, and return 12 outputs. The 54 inputs represents the cube position, while the 12 outputs represents the normalized probabilities of the 12 possible moves.

We will be reusing the informed search algorithm, for data generation. For each cube position, we do an informed search to find a solution. If the solution is not found within the time limit, we take the first step from the model solution. Otherwise, we take the first step from the more efficient solution, between the one given by informed search and the model solution.

The informed search we would use to generate inputs would be:

* Hashmap
* Heuristic: uniformity - coeff * steps_taken
* Coefficient: 3.0

In [1]:
# import necessary libraries for this notebook

from copy import deepcopy
import numpy as np
from sortedcontainers import SortedSet

from time import time
import matplotlib.pyplot as plt
import random

import pickle
import torch
import torch.nn as nn

In [2]:
# define the utils

class Rubik:
    actions = ["L", "L'", "R", "R'", "F", "F'", "B", "B'", "U", "U'", "D", "D'"]
    
    def apply_action(state, action):
        """Applies the action on the state and returns a new state.
        Both `state` and returned state must be numpy array of 6 x 3 x 3 that represents a state of a Rubik's cube.
        Action must be one of "L", "L'", "R", "R'", "F", "F'", "B", "B'", "U", "U'", "D", "D'".

        First, deepcopy the initial state, and then call the respective functions that mutates the copied state.
        Then, return the copied state.
        """

        state = deepcopy(state)
        match action:
            case "L":
                return Rubik.rotate_left_clockwise(state)
            case "L'":
                return Rubik.rotate_left_anticlockwise(state)
            case "R":
                return Rubik.rotate_right_clockwise(state)
            case "R'":
                return Rubik.rotate_right_anticlockwise(state)
            case "F":
                return Rubik.rotate_front_clockwise(state)
            case "F'":
                return Rubik.rotate_front_anticlockwise(state)
            case "B":
                return Rubik.rotate_back_clockwise(state)
            case "B'":
                return Rubik.rotate_back_anticlockwise(state)
            case "U":
                return Rubik.rotate_up_clockwise(state)
            case "U'":
                return Rubik.rotate_up_anticlockwise(state)
            case "D":
                return Rubik.rotate_down_clockwise(state)
            case "D'":
                return Rubik.rotate_down_anticlockwise(state)
            case _:
                raise ValueError(f"Unrecognised action {action}")

    def rotate_face_clockwise(face):
        temp_corner = face[0][0]
        face[0][0] = face[2][0]
        face[2][0] = face[2][2]
        face[2][2] = face[0][2]
        face[0][2] = temp_corner

        temp_side = face[0][1]
        face[0][1] = face[1][0]
        face[1][0] = face[2][1]
        face[2][1] = face[1][2]
        face[1][2] = temp_side

    def rotate_face_anticlockwise(face):
        temp_corner = face[0][0]
        face[0][0] = face[0][2]
        face[0][2] = face[2][2]
        face[2][2] = face[2][0]
        face[2][0] = temp_corner

        temp_side = face[0][1]
        face[0][1] = face[1][2]
        face[1][2] = face[2][1]
        face[2][1] = face[1][0]
        face[1][0] = temp_side

    def rotate_left_clockwise(state):
        temp1, temp2, temp3 = state[0, :, 0]
        state[0, :, 0] = state[5, :, 0]
        state[5, :, 0] = state[4, :, 0]
        state[4, :, 0] = state[2, :, 0]
        state[2, :, 0] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[1])
        return state

    def rotate_left_anticlockwise(state):
        temp1, temp2, temp3 = state[0, :, 0]
        state[0, :, 0] = state[2, :, 0]
        state[2, :, 0] = state[4, :, 0]
        state[4, :, 0] = state[5, :, 0]
        state[5, :, 0] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[1])
        return state

    def rotate_right_clockwise(state):
        temp1, temp2, temp3 = state[0, :, 2]
        state[0, :, 2] = state[2, :, 2]
        state[2, :, 2] = state[4, :, 2]
        state[4, :, 2] = state[5, :, 2]
        state[5, :, 2] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[3])
        return state

    def rotate_right_anticlockwise(state):
        temp1, temp2, temp3 = state[0, :, 2]
        state[0, :, 2] = state[5, :, 2]
        state[5, :, 2] = state[4, :, 2]
        state[4, :, 2] = state[2, :, 2]
        state[2, :, 2] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[3])
        return state

    def rotate_front_clockwise(state):
        temp1, temp2, temp3 = state[0, 2, :]
        state[0, 2, :] = state[1, :, 2][::-1]
        state[1, :, 2][::-1] = state[4, 0, :][::-1]
        state[4, 0, :][::-1] = state[3, :, 0]
        state[3, :, 0] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[2])
        return state

    def rotate_front_anticlockwise(state):
        temp1, temp2, temp3 = state[0, 2, :]
        state[0, 2, :] = state[3, :, 0]
        state[3, :, 0] = state[4, 0, :][::-1]
        state[4, 0, :][::-1] = state[1, :, 2][::-1]
        state[1, :, 2][::-1] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[2])
        return state

    def rotate_back_clockwise(state):
        temp1, temp2, temp3 = state[0, 0, :]
        state[0, 0, :] = state[3, :, 2]
        state[3, :, 2] = state[4, 2, :][::-1]
        state[4, 2, :][::-1] = state[1, :, 0][::-1]
        state[1, :, 0][::-1] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[5])
        return state

    def rotate_back_anticlockwise(state):
        temp1, temp2, temp3 = state[0, 0, :]
        state[0, 0, :] = state[1, :, 0][::-1]
        state[1, :, 0][::-1] = state[4, 2, :][::-1]
        state[4, 2, :][::-1] = state[3, :, 2]
        state[3, :, 2] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[5])
        return state

    def rotate_up_clockwise(state):
        temp1, temp2, temp3 = state[1, 0, :]
        state[1, 0, :] = state[2, 0, :]
        state[2, 0, :] = state[3, 0, :]
        state[3, 0, :] = state[5, 2, :][::-1]
        state[5, 2, :][::-1] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[0])
        return state

    def rotate_up_anticlockwise(state):
        temp1, temp2, temp3 = state[1, 0, :]
        state[1, 0, :] = state[5, 2, :][::-1]
        state[5, 2, :][::-1] = state[3, 0, :]
        state[3, 0, :] = state[2, 0, :]
        state[2, 0, :] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[0])
        return state

    def rotate_down_clockwise(state):
        temp1, temp2, temp3 = state[1, 2, :]
        state[1, 2, :] = state[5, 0, :][::-1]
        state[5, 0, :][::-1] = state[3, 2, :]
        state[3, 2, :] = state[2, 2, :]
        state[2, 2, :] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[4])
        return state

    def rotate_down_anticlockwise(state):
        temp1, temp2, temp3 = state[1, 2, :]
        state[1, 2, :] = state[2, 2, :]
        state[2, 2, :] = state[3, 2, :]
        state[3, 2, :] = state[5, 0, :][::-1]
        state[5, 0, :][::-1] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[4])
        return state

    def turn_tuple(state):
        return tuple(state.reshape(54))

    def is_terminal(state):
        centers = state[:, 1, 1]
        return np.all(state == centers[:, None][:, None])

    def turn_numpy(state):
        return np.array(state).reshape((6, 3, 3))

def heuristic(state, steps_taken):
    def face_heuristic(face):
        count = 0
        for i in range(3):
            for j in range(3):
                if i < 2 and face[i][j] == face[i + 1][j]:
                    count += 1
                if j < 2 and face[i][j] == face[i][j + 1]:
                    count += 1
        return count
    return sum([face_heuristic(face) for face in state]) - 3.0 * steps_taken

class Node:
    def __init__(self, state, steps_taken=0, parent=None, action=None):
        """Initialises a node.
        `state`: A numpy array of dimension 6 x 3 x 3
        `steps_taken`: The number of steps taken from the initial state.
        `parent`: An instance of `Node` that contains the previous state.
        `action`: The action that transitions the previous state to this state.
        """
        self.state = Rubik.turn_tuple(state)
        self.steps_taken = steps_taken
        self.parent = parent
        self.action = action
        self.h_value = heuristic(state, steps_taken)

    def __lt__(self, node):
        """Determines the priority of this node compared to another node.
        This determines the position of this node in the search frontier.
        """
        return self.h_value < node.h_value

    def __hash__(self):
        """Determines the hash value of this node.
        This enables the hashing algorithm of the set of explored states.
        Nodes with the same states must have the same hash values.
        """
        return hash(self.state)

    def __eq__(self, other):
        """Determines the equality of two nodes.
        This enables the hashing algorithm of the set of explored states.
        Since we do not want to re-explore the same states, any two nodes
        with the same state must be equal, regardless of the values of other fields.
        """
        return self.state == other.state
        
    def get_numpy_state(self):
        """Get the numpy array that represents the state contained in this node.
        The numpy array must be of dimension 6 x 3 x 3.
        """
        return Rubik.turn_numpy(self.state)

def reverse_move(move):
    if len(move) == 2:
        return move[0]
    else:
        return move + "'"

def get_terminal_state():
    return np.array([
        [[i for k in range(3)] for j in range(3)] for i in range(6)
    ])

def generate_puzzle(scramble):    
    puzzle = get_terminal_state()
    for action in scramble:
        puzzle = Rubik.apply_action(puzzle, action)
    scramble.reverse()
    solution = [reverse_move(move) for move in scramble]
    return puzzle, solution

def solve(state, time_limit):
    step_limit = 26
    start_time = time()
    first_node = Node(state)
    frontier = SortedSet([first_node])
    explored = {
        first_node: first_node.steps_taken,
    }

    while len(frontier) != 0:
        curr_node = frontier.pop()
        curr_state = curr_node.get_numpy_state()
        
        for action in Rubik.actions:
            child_state = Rubik.apply_action(curr_state, action)
            if Rubik.is_terminal(child_state):
                actions = [action]
                while curr_node.parent is not None:
                    actions.append(curr_node.action)
                    curr_node = curr_node.parent
                actions.reverse()
                return actions, time() - start_time

            if curr_node.steps_taken == step_limit - 1:
                continue

            child_node = Node(
                child_state,
                steps_taken=curr_node.steps_taken + 1,
                parent=curr_node,
                action=action,
            )
            if child_node in explored:
                if explored[child_node] > child_node.steps_taken:
                    explored[child_node] = child_node.steps_taken
                    frontier.add(child_node)
            else:
                explored[child_node] = child_node.steps_taken
                frontier.add(child_node)

        if time() - start_time > time_limit:
            return None, time_limit

## Define the data generator

The data generator would randomly generate a scramble, and generate the puzzle and the model solution for it.
The puzzle would then go through the informed search to see if there is a more efficient solution.
A few things to take note:
1. The scramble generator must be such that it does not revert its previous action. This means, for each current move, we prevent the next move from being a reverse of it. There are many more ways that the next move can revert the current moves, but we can only go so far to prevent some of such cases.
2. The final input generator would compare the solution returned by informed search and the model solution.

In [3]:
def generate_scramble(num_of_moves):
    scramble = []
    current_reverse_move = None
    for _ in range(num_of_moves):
        move = random.choice(Rubik.actions)
        while move == current_reverse_move:
            move = random.choice(Rubik.actions)
        current_reverse_move = reverse_move(move)
        scramble.append(move)
    return scramble

In [4]:
test_5_move_scramble = generate_scramble(5)
test_10_move_scramble = generate_scramble(10)
test_15_move_scramble = generate_scramble(15)

print(test_5_move_scramble)
print(test_10_move_scramble)
print(test_15_move_scramble)

assert all([move in Rubik.actions for move in test_5_move_scramble]) and len(test_5_move_scramble) == 5
assert all([move in Rubik.actions for move in test_10_move_scramble]) and len(test_10_move_scramble) == 10
assert all([move in Rubik.actions for move in test_15_move_scramble]) and len(test_15_move_scramble) == 15

["L'", 'R', "D'", 'R', 'L']
['F', 'D', "R'", 'D', "U'", 'L', 'D', "R'", 'U', 'R']
["R'", "L'", 'F', 'U', 'B', 'F', 'U', 'B', "F'", 'B', 'U', 'U', 'F', 'B', "R'"]


In [4]:
def generate_input(num_of_moves, time_limit, debug=False):
    scramble = generate_scramble(num_of_moves)
    puzzle, model_solution = generate_puzzle(scramble)
    if debug:
        print(f"Generating solution for {model_solution}")
    puzzle_tuple = Rubik.turn_tuple(puzzle)
    
    solution, time_spent = solve(puzzle, time_limit)
    if debug:
        print(f"Solution found: {solution}, in {time_spent}")
    if solution is None:
        return puzzle_tuple, model_solution[0]
    elif len(solution) < len(model_solution):
        return puzzle_tuple, solution[0]
    else:
        return puzzle_tuple, model_solution[0]

In [10]:
test_time_limit = 5
print(generate_input(5, test_time_limit, debug=True))
print(generate_input(10, test_time_limit, debug=True))
print(generate_input(15, test_time_limit, debug=True))

Generating solution for ['U', 'F', "B'", 'U', 'D']
Solution found: ['U', "B'", 'F', 'D', 'U'], in 0.0157468318939209
((5, 0, 5, 3, 0, 3, 2, 0, 2, 1, 5, 3, 0, 1, 0, 0, 2, 0, 0, 5, 0, 1, 2, 3, 1, 2, 3, 1, 2, 3, 4, 3, 4, 4, 5, 4, 5, 1, 2, 4, 4, 4, 5, 1, 2, 3, 5, 1, 3, 5, 1, 4, 2, 4), 'U')
Generating solution for ["U'", "D'", 'U', 'D', "F'", "R'", "D'", "F'", "D'", 'R']
Solution found: ["F'", "R'", "D'", "F'", "D'", 'R'], in 0.04270434379577637
((0, 0, 2, 0, 0, 2, 3, 4, 4, 1, 1, 2, 1, 1, 4, 3, 3, 5, 4, 1, 1, 5, 2, 2, 3, 3, 2, 5, 0, 0, 1, 3, 3, 4, 3, 3, 0, 0, 1, 2, 4, 4, 2, 4, 4, 0, 2, 5, 5, 5, 5, 5, 5, 1), "F'")
Generating solution for ["F'", 'B', 'F', 'L', 'F', 'U', 'R', "U'", 'F', "U'", 'B', "R'", 'U', 'B', 'F']
Solution found: ['B', 'L', 'F', 'U', 'R', "U'", 'F', "U'", 'B', "R'", 'U', 'B', 'F'], in 3.7969799041748047
((2, 1, 0, 4, 0, 4, 1, 5, 3, 0, 3, 0, 2, 1, 1, 2, 3, 4, 5, 1, 4, 4, 2, 2, 2, 4, 4, 5, 5, 3, 0, 3, 3, 1, 1, 4, 3, 2, 2, 5, 4, 2, 1, 0, 1, 0, 5, 5, 3, 5, 0, 3, 0, 5), 'B')


In [5]:
def generate_data(initial_data={}, rounds=10, time_limit=5, debug=False):
    action_map = { "L": 0, "L'": 1, "R": 2, "R'": 3, "F": 4, "F'": 5, "B": 6, "B'": 7, "U": 8, "U'": 9, "D": 10, "D'": 11 }
    data = initial_data
    for _ in range(rounds):
        num_of_moves = random.randint(1, 32)
        puzzle, action = generate_input(num_of_moves, time_limit=time_limit, debug=debug)
        data[puzzle] = action_map[action]
    return data

In [15]:
print(generate_data(debug=True))

Generating solution for ["B'", 'U', 'B', 'U', "B'", 'F', "U'", "B'", "R'", "D'", "B'", "U'", 'D', "U'", "B'", "F'", "L'", "B'", 'F', "L'"]
Solution found: None, in 5
Generating solution for ['D', "U'", "L'", "R'", "D'", 'R', 'L', "B'", 'U', "R'", 'F']
Solution found: ['D', "U'", "R'", "L'", "D'", 'L', 'R', "B'", 'U', "R'", 'F'], in 1.446103811264038
Generating solution for ['D', "U'", "L'", 'D', 'F', "R'", 'D', 'L', 'D', "R'", "U'", "U'", "D'", "F'", 'D', 'B', "F'", 'D', 'B', 'U', 'D', "R'", 'D', 'B', 'B', "F'", "F'", 'U', 'U', 'U', "B'", "D'", 'B', "D'"]
Solution found: None, in 5
Generating solution for ['D', 'U', 'D', 'D', "F'", "F'", 'R', "B'", 'F', 'F', 'R', 'D', 'U', 'B', 'B', "R'", "R'", 'F', "D'", "B'", "B'", "R'", 'F', 'L', 'U', 'D', 'F', 'L', "D'", "B'", 'F', "B'", 'R', 'R']
Solution found: None, in 5
Generating solution for ['D', 'F', 'L']
Solution found: ['D', 'F', 'L'], in 0.015633344650268555
Generating solution for ['L', "D'", 'U', 'U', "L'", "B'", 'F', "D'", 'U', 'D', "

In [9]:
def generate_data_main(filename="data/data.pkl", rounds=10, time_limit=5, debug=True):
    try:
        with open(filename, "rb") as f:
            data = pickle.load(f)
    except FileNotFoundError:
        data = {}

    print(f"Initial data size: {len(data)}")
    if debug:
        print(f"initial data: {data}")

    data = generate_data(initial_data=data, rounds=rounds, time_limit=time_limit, debug=debug)
    print(f"Final data size: {len(data)}")
    with open(filename, "wb") as f:
        pickle.dump(data, f)

In [11]:
generate_data_main(rounds=1000, debug=False)

Initial data size: 1944
Final data size: 2888
